<a href="https://colab.research.google.com/github/RahnamaRad/ALO/blob/master/fast_ALO_linear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import shutil

folder_path = '/content/fast-ALO'  # Replace 'my_folder' with the path to your directory

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print("Folder removed successfully")
else:
    print("Folder does not exist")


Folder does not exist


In [2]:
import os
import shutil

!git clone https://github.com/RahnamaRad/fast-ALO

# Change the current directory to the cloned repository
os.chdir('fast-ALO')

Cloning into 'fast-ALO'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 32 (delta 16), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (32/32), 2.80 MiB | 13.81 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.stats import laplace, norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import cg, LinearOperator, inv
from time import time  # Make sure to import the time function

def create_sparse_D_matrix(p):
    # Diagonal elements
    diagonal = np.ones(p-1)
    # Off-diagonal elements
    off_diagonal = -1 * np.ones(p-1)
    # Create sparse matrix with diagonal and off-diagonal
    D = diags([diagonal, off_diagonal], [0, 1], shape=(p-1, p))
    return D

# Set the seed for reproducibility
np.random.seed(1)
# Parameters
logp_       = np.arange(0,4)
plogstep    = 2
p_          = 500* plogstep**(logp_)
p_          = p_.astype(int)
delta       = 1 # n/p
lm          = 10
S           = 250  # Number of iterations/samples
# Vectors of zeros
ALO           = []  # Creates a column vector of zeros of length m
ALO_fast      = []   # Same as above
TRAIN         = []  # Same as above
timeALO       = [] # Creates a column vector of zeros of length m
timeALO_fast  = []   # Same as above
timeTRAIN     = []  # Same as above

# Define the matrix-vector product function outside the loop
def matvec(v, X, D, lambda_reg):
    return X.T @ (X @ v) + lambda_reg * (D.T @ (D @ v))


# Generate data and fit model
for i, p in enumerate(p_):
    n = int(p * delta)
    X = np.random.normal(0, 1/np.sqrt(n), (n, p))
    D = create_sparse_D_matrix(p)
    m = D.shape[0]
    # Arrays to store results
    b_s1        = np.zeros((p, S))  # to store results of X.T @ e_s1
    b_s2        = np.zeros((p, S))  # to store results of D.T @ e_s2
    u_s         = np.zeros((p, S))
    Hii         = np.zeros((p, lm))
    Hii_fast    = np.zeros((p, lm))
    for s in range(S):
        # Compute b_s1 and b_s2
        # Sample e_s1, e_s2 from N(0, I)
        e_s1 = np.random.normal(0, 1, n)
        e_s2 = np.random.normal(0, 1, m)
        b_s1[:, s] = X.T @ e_s1  # Compute X^T @ e_s1
        b_s2[:, s] = D.T @ e_s2  # Compute D^T @ e_s2
        #if i > 0:
        #if s % 100 == 99:  # Print every 100 iterations of s
         #   print(f"p= {p:4d}| s={s+1:3d}| df={df:.2f}, lmb={lambdaS[0]*n:.1f}| lo={lo[s, i]:.2f}| outErr={outErr[s, i]:.2f}| MSE ={MSE[i]:.3f} | scaling = {a:.2f} | R2_MSE = {R2:.2f}| res=  {np.mean(np.abs(res)):.4f}|r1 = {np.mean(np.abs(r1)):.4f}| r2 = {np.mean(np.abs(r2)):.4f}")

    # Loop over different lambda values
    for lambda_reg in lambdas:
        # Define shape of A for the LinearOperator with the current lambda
        A = LinearOperator((p, p), matvec=lambda v: matvec(v, X, D, lambda_reg))
        b = b_s1 + lambda_reg * b_s2
        # Start timing - TRAIN
        tstart = time()
        # Solve for y using conjugate gradient
        z, exit_code = cg(A, b, x0=z, tol=tol, maxiter=maxiter)
        # End timing
        fit_time = time() - tstart
        timeTRAIN.append(fit_time)  # Store the fitting time
        mse = np.mean((r - (X @ z))**2)
        TRAIN.append(mse)

    # Start timing - ALO
    tstart = time()
    diag_inv = np.zeros((n2,1))
    for i in range(n2):
        # Create the i-th unit vector in sparse format
        e_i = csr_matrix((np.array([1]), ([i], [0])), shape=(n2,1))
        e_i = e_i.toarray().ravel()
        # Solve A x = e_i
        # Since e_i is a 1xN matrix, we need to flatten it to make it a proper vector
        ae, exit_code = cg(A, e_i, tol=tol, maxiter=maxiter)
        if exit_code == 0:
            # The i-th diagonal element of A^{-1} is the i-th element of ae
            diag_inv[i] = ae[i]
            e = A @ ae
            #if (np.sum(abs(A @ ae- e_i)) > 0.001):
            #    print(np.sum(abs(A @ ae- e_i)))
        else:
            raise Exception(f"CG did not converge for the {i}-th unit vector")
    hh = np.zeros((T,1))
    for t in range(T):
        hh[t] = diag_inv[I[t]]
    # End timing
    alo_time = time() - tstart
    timeALO.append(alo_time)  # Store the fitting time
    alo     = np.mean(((r - (X @ z) ) / (1 - hh)) **2)
    ALO.append(alo)
    Hii[:, lmi] = diag_inv.flatten()

    # Start timing - fast ALO
    tstart = time()
    b_s = b_s1 +  np.sqrt(lambda_reg) * b_s2
    # Solve the system A x = b for each column of b_s
    for s in range(S):
        u_, info = cg(A, b_s[:, s], tol=tol, x0=u_s[:, s])
        if info == 0:
            u_s[:, s] = u_
        else:
            print(f"CG did not converge for column {s}")
    #XU = X @ u_s
    #Z =  (u_s @ u_s.T) / S
    # Step 1: Center the data
    means = np.mean(u_s, axis=1, keepdims=True)
    u_s_centered = u_s - means
    # Step 2: Compute the covariance matrix
    Z =  np.dot(u_s_centered, u_s_centered.T) / (S - 1)
    #for l in range(4):
     #   Z  = 2*Z - Z @ (A @ Z)
    diag_inv_ = Z.diagonal()
    Hii_fast[:, lmi] = diag_inv_
    Z  = (X @ Z) @ X.T
    hh_fast = np.diag(Z)

    #print(hh_fast)
    #np.var(XU, axis=1, ddof=1)  # ddof=1 uses (S-1) in the denominator
    # End timing
    alo_fast_time = time() - tstart
    timeALO_fast.append(alo_fast_time)  # Store the fitting time
    alo_fast     = np.mean(((r - (X @ z)) / (1 - hh_fast)) **2)
    ALO_fast.append(alo_fast)
    print(f"lambda = {lambda_reg:.2f}| alo={alo:.2f}| alo time ={alo_time:.0f}| alo_fast={alo_fast:.2f}| alo_fast time ={alo_fast_time:.0f}| mse={mse:.2f}| fit time = {fit_time:.2f}| h ={np.mean(hh):.2f}| max_err ={np.max(abs(diag_inv_ - diag_inv)):.2f}| mean_err ={np.mean(abs(diag_inv_ - diag_inv)):.2f}")
    lmi = lmi + 1

# Ensure all arrays are numpy arrays for easier handling
fit_times = np.array(timeTRAIN)
tt=np.mean(fit_times)

    # Collect results for the DataFrame
    MSE_SE[i] = np.std((lo[:, i] - outErr[:, i]) ** 2) / np.sqrt(MCMCsamples)
    results.append({
        'delta': delta,
        'rho': rho,
        'alpha_elnet': alpha_elnet,
        'n': n,
        'p': p,
        'MSE': MSE[i],
        'bias': bias[i],
        'var': var[i],
        'MSE_SE': MSE_SE[i]
    })

ls_mse_fit = sm.OLS(np.log(MSE), sm.add_constant(np.log(p_))).fit()
a_mse = ls_mse_fit.params[1]
R2_mse = ls_mse_fit.rsquared

ls_var_fit = sm.OLS(np.log(var), sm.add_constant(np.log(p_))).fit()
a_var = ls_var_fit.params[1]
R2_var = ls_var_fit.rsquared

bias2 = bias**2
ls_bias2_fit = sm.OLS(np.log(bias2), sm.add_constant(np.log(p_))).fit()
a_bias2 = ls_bias2_fit.params[1]
R2_bias2 = ls_bias2_fit.rsquared

print(f" MSE: scaling = {a_mse:.2f} | R2 = {R2_mse:.2f}")
print(f" var: scaling = {a_var:.2f} | R2 = {R2_var:.2f}")
print(f" bias2: scaling = {a_bias2:.2f} | R2 = {R2_bias2:.2f}")

# Calculate MSE standard error
MSE_SE = np.std((lo - outErr) ** 2, axis=0) / np.sqrt(MCMCsamples)
print("Mean squared errors:", np.mean((lo - outErr) ** 2, axis=0))
print("MSE standard errors:", MSE_SE)

# Convert results to a NumPy array
results_array = np.array(results)

from google.colab import files

filename = f'linearELNET_{"pos" if pos else "neg"}.npy'

if alpha_elnet == 1:
  filename = f'linearLASSO_{"pos" if pos else "neg"}.npy'

np.save(filename, results_array, allow_pickle=True)

files.download(filename)
